# DECO3801 - Shapez.AI Project
09:21:04 Friday September 20, 2024

Student name:    &emsp;Cain Bruhn-Tanzer \
Student ID:      &emsp;&emsp;&ensp;s4535376 \
GitHub repo:     &emsp;&ensp; https://github.com/CainOCE/Shapez.ai


Static Goal List

In [ ]:
/**
 * need the list of goals
 * need a way to send the static required amount i dont know if we
 * can because the way we have it at the moment we have it we only
 * send through it once.
 */

// CuCuCuCu
// ----CuCu
// RuRuRuRu
// RuRU----
// Cu----Cu
// Cu------
// CrCrCrCr
// RbRb----
// CpCpCpCp
// ScScScSc
// CgScScCg
// CbCbCbRb:CwCwCwCw
// RpRpRpRp:CwCwCwCw
// --Cg----:--Cr----
// SrSrSrSr:CyCyCyCy
// SrSrSrSr:CyCyCyCy:SwSwSwSw
// CbRbRbCb:CwCwCwCw:WbWbWbWb
// Sg----Sg:CgCgCgCg:--CyCy--
// CpRpCp--:SwSwSwSw
// RuCw--Cw:----Ru--
// CrCwCrCw:CwCrCwCr:CrCwCrCw:CwCrCwCr
// Cg----Cr:Cw----Cw:Sy------:Cy----Cy
// CcSyCcSy:SyCcSyCc:CcSyCcSy
// CcRcCcRc:RwCwRwCw:Sr--Sw--:CyCyCyCy
// Rg--Rg--:CwRwCwRw:--Rg--Rg
// CbCuCbCu:Sr------:--CrSrCr:CwCwCwCw